<a href="https://colab.research.google.com/github/tomkeane07/rugby_tracker/blob/submitted_thesis/Copy_of_RugbyFieldRegistration(6).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install  opencv-python==3.4.2.17
# !pip install  opencv-contrib-python==3.4.2.17
%matplotlib inline

In [ ]:
from google.colab import drive
from google.colab.patches import cv2_imshow
# drive.mount('/content/drive')
import matplotlib.pyplot as plt
import imutils
import cv2
import numpy as np
import copy
import sys
from sklearn.cluster import AgglomerativeClustering
try:
  from pyfld import FastLineDetector
except:
  ! pip install pyfld
  from pyfld import FastLineDetector
import collections
import pandas as pd

from moviepy.editor import *
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

from os import linesep
from scipy.spatial import distance
from itertools import chain
import matplotlib
import matplotlib.pyplot as plt
import operator

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
try:
  model
except:
  ! git clone https://github.com/ultralytics/yolov5  # clone
  ! pip install -r ./yolov5/requirements.txt  # install
  import torch
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
  # model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

In [ ]:
class ImgMorphOperator:
  def __init__(self):
    # self.kernel2 = np.ones((2,2), np.uint8)
    self.kernel3 = np.ones((3,3), np.uint8)
    # self.kernel4 = np.ones((4,4), np.uint8)
    # self.kernel5 = np.ones((5,5), np.uint8)
    # self.kernel6 = np.ones((6,6), np.uint8)
    self.kernel8 = np.ones((8,8), np.uint8)
    # self.kernel8x1 = np.ones((8,1), np.uint8)
    self.kernel10 = np.ones((10,10), np.uint8)
    self.kernel15 = np.ones((15,15), np.uint8)
    # self.kernel20 = np.ones((20,20), np.uint8)
    self.kernel30 = np.ones((30,30), np.uint8)
    self.kernel50 = np.ones((50,50), np.uint8)
    # self.kernel1x100 = np.ones((1,100), np.uint8)
    # self.kernel1x200 = np.ones((1,200), np.uint8)
    # self.kernel1x500 = np.ones((1,500), np.uint8)
    self.kernel5x50 = np.ones((5,50), np.uint8)
    self.kernel2x15 = np.ones((2,15), np.uint8)
    self.kernel15x1 = np.ones((15,1), np.uint8)
    self.kernel15x2 = np.ones((15,2), np.uint8)
    self.kernel5x30 = np.ones((5,30), np.uint8)
    self.kernel1x15 = np.ones((1,15), np.uint8)
    self.kernel1x30 = np.ones((1,30), np.uint8)
    self.kernel60x1 = np.ones((60,1), np.uint8)
    self.kernel75 = np.ones((75, 75), np.uint8)
    self.kernel1x85 = np.ones((1, 85), np.uint8)
    # self.kernel85 = np.ones((85, 85), np.uint8)
    # self.kernel100 = np.ones((100,100), np.uint8)


  def run(self, mask):
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel8)
    # mask = cv2.dilate(mask, self.kernel30, 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel75)
    mask = cv2.dilate(mask, self.kernel15x1, 1)
    mask = cv2.dilate(mask, self.kernel15x1, 1)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel75)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.erode(mask, self.kernel5x30, 1)
    mask = cv2.dilate(mask, self.kernel15x1, 1)
    mask = cv2.dilate(mask, self.kernel8, 1)
    # mask = cv2.erode(mask, self.kernel1x30, 1)
    # mask = cv2.erode(mask, self.kernel1x30, 1)
    # mask = cv2.erode(mask, self.kernel1x30, 1)
    # mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel30)
    return mask

  def run1(self, mask):
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, self.kernel8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel15)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel15)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.kernel1x30)
    mask = cv2.dilate(mask, self.kernel5x30, 1)
    mask = cv2.dilate(mask, self.kernel8, 1)
    return mask

In [ ]:
class PitchIsolator:
  def __init__(self, imgMorphOperator=None):
    self.imo = ImgMorphOperator() if imgMorphOperator==None else imgMorphOperator
  # (70,115,85), (80,255,180)

  def findPitch(self, image, range=[(0, 95, 55), (115, 255, 170)], run='0'):
    mask = cv2.inRange(image, lowerb=range[0], upperb=range[1])
    yg = cv2.bitwise_not(cv2.inRange(image, lowerb=(65,145,125), upperb=(92,175,155)))
    dg = cv2.bitwise_not(cv2.inRange(image, lowerb=(65,93,56), upperb=(77,115,100)))
    # yg = cv2.bitwise_not(cv2.inRange(image, lowerb=(55,135,115), upperb=(100,185,165)))
    # dg = cv2.bitwise_not(cv2.inRange(image, lowerb=(55,83,46), upperb=(77,115,100)))
    mask = cv2.bitwise_and(mask, yg)
    mask = cv2.bitwise_and(mask, dg)
    if run=='0':
      # cv2_imshow(dg)
      # cv2_imshow(mask)
      mask = self.imo.run(mask)
      mask = self.cropImg(mask)
    elif run=='1':
      mask = self.imo.run1(mask)
    return mask

  def removeNonPitch(self, image, mask):
    # apply mask
    return cv2.bitwise_and(image, image, mask=mask)

  def isoLines(self, image, range=[(110,150,150), (130,255,255)]):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, lowerb=range[0], upperb=range[1])
    return cv2.bitwise_and(image, image, mask=mask)


  def cropImg(self, image,tol=1):
      # img is 2D image data
      # tol  is tolerance
      self.cropMask = image>tol
      return image[np.ix_(self.cropMask.any(1),self.cropMask.any(0))]


  def blockBillboards(self, image):
    red=          cv2.bitwise_not(cv2.inRange(image, (85, 46, 150), (120, 135, 180)))
    pink      =         cv2.bitwise_not(cv2.inRange(image, (200, 60, 150), (255, 140, 245)))
    navy        =          cv2.bitwise_not(cv2.inRange(image, (250, 40, 5), (255, 135, 52)))
    cream     =          cv2.bitwise_not(cv2.inRange(image, (30, 90, 120), (100, 150, 255)))
    guinBlack   =    cv2.bitwise_not(cv2.inRange(image, (0, 0, 0), (255, 120, 100)))
    bbyblue    =      cv2.bitwise_not(cv2.inRange(image, (250, 165, 25), (255, 255, 155)))
    yellow    =      cv2.bitwise_not(cv2.inRange(image, (15, 240, 240), (60, 255, 255)))
    yellowGrass =     cv2.bitwise_not(cv2.inRange(image, (115,215,190), (150,255,225)))
    mask = red & pink & navy & cream & guinBlack & bbyblue & yellow & yellowGrass
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, self.imo.kernel8)
    # cv2_imshow(mask)
    image= cv2.bitwise_and(image,  image, mask=mask)
    # cv2_imshow(image)
    return image


  def lineImgPrep(self, image):
    image = cv2.convertScaleAbs(image, alpha=1.25, beta=15)
    blackenBlack = cv2.inRange(image, (0,0,0), (20,20,20))
    # image = cv2.bitwise_and(image,  image, mask=blackenBlack)
    mwrongs = cv2.bitwise_not(cv2.inRange(image, (130, 145, 125), (170, 210, 190)))
    mwrongs = cv2.bitwise_not(cv2.add(cv2.bitwise_not(mwrongs), blackenBlack))
    # mwrongs=cv2.morphologyEx(mwrongs, cv2.MORPH_OPEN, self.imo.kernel15)
    # mwrongs=cv2.morphologyEx(mwrongs, cv2.MORPH_CLOSE, self.imo.kernel8)
    mwrongs=cv2.morphologyEx(mwrongs, cv2.MORPH_CLOSE, self.imo.kernel1x30)
    # sys.exit(0)
    image = cv2.bitwise_and(image,  image, mask=mwrongs)
    image=self.blockBillboards(image)
    # # lineMask = cv2.inRange(image, (110, 175, 135), (205, 250, 205))
    # # lineMask=cv2.cv2.erode(lineMask, self.imo.kernel8, 1)
    # lineMask = cv2.inRange(image, (110, 175, 135), (195, 250, 195))
    # lineMask=cv2.morphologyEx(lineMask, cv2.MORPH_CLOSE, self.imo.kernel8)
    # lineMask=cv2.dilate(lineMask, self.imo.kernel3)
    # image = cv2.bitwise_and(image,  image, mask=lineMask)
    # sys.exit()
    return image


  def isolatePitch(self, image):
    mask = self.findPitch(cv2.convertScaleAbs(copy.deepcopy(image), alpha=1.5, beta=-30))
    playersFrame = self.removeNonPitch(image[np.ix_(self.cropMask.any(1),self.cropMask.any(0))], mask)
    mask = self.findPitch(cv2.convertScaleAbs(playersFrame, alpha=1.5, beta=-30), run='1')
    shi=np.shape(image)
    linesFrame = self.removeNonPitch(image[np.ix_(self.cropMask.any(1),self.cropMask.any(0))], mask)
    linesFrame = self.lineImgPrep(linesFrame)
    # print(np.shape(linesFrame))
    linesFrame = cv2.resize(linesFrame, (2560,800))
    playersFrame = cv2.resize(playersFrame, (2560,800))
    return playersFrame, linesFrame

In [ ]:
class LineMerger:
  def __init__(self, lco):
    self.lco = lco
    self.cornerPoints = np.empty(shape=(0,3))

  def lengthPrioritizeEqs(self, lineGroup):
    eqs=[]
    longLn = np.max(np.append(lineGroup[:,3],1e-10))
    for ln in lineGroup:
      factor=int(4*ln[3]/(longLn+1))
      fx=ln[2]
      eqs.append(fx)
      for i in range(factor):
        eqs.append(fx)
    return eqs


  def getCornerPoints(self, linesID):
    self.CornerPoints = {'ovrMax':100, 'ovrMin':100}
    for lnType in np.unique(linesID[:,1]):
      # print(lnType)
      indices = np.where((np.array(linesID[:,1])==lnType))[0]
      ys = [row[0][[1,3]] for row in linesID[indices]]
      ys = list(chain.from_iterable(ys))
      # print(ys)
      minIndex, maxIndex = np.argmin(ys), np.argmax(ys)
      if ys[minIndex] < self.CornerPoints['ovrMin']:
        self.CornerPoints['ovrMin'] = ys[minIndex]
      if ys[minIndex] > self.CornerPoints['ovrMax']:
        self.CornerPoints['ovrMax'] = ys[minIndex]
      minPoint = [linesID[indices][minIndex//2][0][minIndex%2], linesID[indices][minIndex//2][0][minIndex%2+1]]
      maxPoint = [linesID[indices][maxIndex//2][0][maxIndex%2], linesID[indices][maxIndex//2][0][maxIndex%2+1]]
      self.CornerPoints[lnType] = {'minPoint':minPoint, 'maxPoint':maxPoint}

  def refineLines(self, lineGroup, minLenFactor=0.3):
    delLinesIndex = []
    longestLen  = np.max([self.lco.dist(self.lco.points(x)) for x in lineGroup[:,0]])
    shortestLen = np.min([self.lco.dist(self.lco.points(x)) for x in lineGroup[:,0]])
    defaultDist = longestLen*minLenFactor
    for i in range(len(lineGroup)):
        ln =  lineGroup[i][0]
        pt1,pt2 = self.lco.points(ln)
        dist = self.lco.dist((pt1,pt2))
        if dist < defaultDist:
          delLinesIndex.append(i)
          # keepLines1+=colLines
    return delLinesIndex

  def remove_outliers_bis(self, arr, k=1.4, lean=''):
    # print(arr)
    delIndex, mask=[], []
    arr = np.array(arr)
    mask = np.ones((arr.shape[0],), dtype=bool)
    if len(arr)<3: return arr,[]
    mu, sigma = np.mean(arr, axis=0), np.std(arr, axis=0, ddof=1)
    if lean=='up':
      mu[-1]-=np.abs(sigma[-1])*0.5
    elif  lean=='down':
      mu[-1]+=np.abs(sigma[-1])*0.5
    nCols=arr.shape[1]
    for j in range(nCols):
        col = arr[:, j]
        keep = np.abs((col[mask] - mu[j]) / sigma[j]) <= k
        mask[mask] &= keep
        # sys.exit(0)
        if len(arr[mask])<=1:
          mask = np.ones((arr.shape[0],), dtype=bool)
          if k > 5:
            print("quit")
            return arr, []
          return self.remove_outliers_bis(arr, k=k*1.5, lean=lean)
        for i in range(len(keep)):
          if not keep[i]:
            delIndex = np.append(delIndex, i)
    delIndex = np.unique(delIndex)
    return arr[mask], delIndex.astype(int)

  def mergeLineType(self, lineGroup, lnType):
    # print("lineGroup", lineGroup)
    # delIndex = self.refineLines(lineGroup)
    # lineGroup = np.delete(lineGroup, delIndex, axis=0)
    eqs=self.lengthPrioritizeEqs(lineGroup)
    if lnType=="halfway":
      k=1.2
      lean=''
    elif lnType=="nearSideLine":
      k=1
      lean=''
    elif lnType=="farSideLine":
      k=1.2
      lean=''
    elif lnType=="tryLineL" or lnType=="tryLineR":
      k=1.2
      lean=''
    elif lnType=="22L" or lnType=="22R":
      k=1.2
      lean=''
    else:
      k=1.2
      lean=''
    print(lnType)
    eqs, delIndex = self.remove_outliers_bis(eqs,k=k, lean=lean)
    # if len(delIndex) > 0:
    i,j=0,0
    while  i <len(delIndex)-1:
      j=delIndex[i]
      if j >= len(lineGroup):break
      lineGroup = np.delete(lineGroup,j, axis=0)
      i+=1
    minmaxX = [np.min(np.array([pts[[0,2]] for pts in lineGroup[:,0]]).flatten()), np.max(np.array([pts[[0,2]] for pts in lineGroup[:,0]]).flatten())]
    minmaxY = [np.min(np.array([pts[[1,3]] for pts in lineGroup[:,0]]).flatten()), np.max(np.array([pts[[1,3]] for pts in lineGroup[:,0]]).flatten())]
    newLine = [
               [minmaxX[0], np.poly1d(np.mean(eqs, axis=0))(minmaxX[0]),
                minmaxX[1], np.poly1d(np.mean(eqs, axis=0))(minmaxX[1])],
                lnType,
                np.mean(eqs, axis=0)
              ]
    return newLine

  def run(self, linesID):
    # print(linesID)
    delIndex = np.where((np.array(linesID)[:,1]=="other")|(np.array(linesID)[:,1]=="UpPitchLine"))[0]
    linesID = np.delete(linesID, delIndex, axis=0)
    # print(linesID)
    self.getCornerPoints(linesID)
    lnTypes = np.unique(linesID[:,1])
    newLines = []
    for lnType in lnTypes:
      # print(lnType)
      self.indices = np.where((np.array(linesID[:,1])==lnType))[0]
      newLine = self.mergeLineType(linesID[self.indices], lnType)
      newLines.append(newLine)
    print(newLines)
    return newLines

In [ ]:
class LineClassifier:
  def __init__(self, lco):
    self.lco = lco


  def lengthPrioritizeLines(self, lnTypes=[]):
    for lnT in lnTypes:
      lineGroup=self.linesID[ np.where(self.linesID[:,1]==lnT)[0] ]
      # print("lnG", lineGroup)
      longLn = np.max(np.append(lineGroup[:,3],1e-10))
      # print(shortLn)
      for ln in self.linesID:
        if ln[1]==lnT:
          factor=int((1.05*ln[3])/(longLn))*3
          # print(factor)
          for i in range(factor):
            self.linesID=np.append(self.linesID, [ln], axis=0)

  @property
  def lineTypeSlopes(self):
    lineTypeSlopes =     {'UpPitchLine': [], 'halfway': [], 'other': [], '22L': [], 'tryLineL':[], 'tryLineR':[], '22R':[], 'nearSideLine':[], 'farSideLine':[]}
    for ln in self.linesID:
      lnType=ln[1]
      lineTypeSlopes[lnType].append(ln[2][0])
    return lineTypeSlopes

  @property
  def lineTypeCs(self):
    lineTypeCs =     {'UpPitchLine': [], 'halfway': [], 'other': [], '22L': [], 'tryLineL':[], 'tryLineR':[], '22R':[], 'nearSideLine':[], 'farSideLine':[]}
    for ln in self.linesID:
      lnType=ln[1]
      lineTypeCs[lnType].append(ln[2][1])
    return lineTypeCs

  @property
  def lineTypeYintercepts(self):
    lineTypeCs =     {'UpPitchLine': [], 'halfway': [], 'other': [], '22L': [], 'tryLineL':[], 'tryLineR':[], '22R':[], 'nearSideLine':[], 'farSideLine':[]}
    for ln in self.linesID:
      lnType=ln[1]
      C=ln[2][1]
      slope=ln[2][0]
      lineTypeCs[lnType].append(-C/slope)
    return lineTypeCs

  @property
  def lineTypeLengthSum(self):
    lnsums = {'UpPitchLine': 0, 'halfway': 0, 'other': 0, '22L': 0, 'tryLineL':0, 'tryLineR':0, '22R':0, 'nearSideLine':0, 'farSideLine':0}
    for ln in self.linesID:
      lnsums[ln[1]] += self.lco.dist(ln=ln[0])
    return lnsums

  @property
  def lineTypeMeanPos(self):
    pos = {'UpPitchLine': [[0,0]], 'halfway': [[0,0]], 'other': [[0,0]], '22L': [[0,0]], 'tryLineL':[[0,0]], 'tryLineR':[[0,0]], '22R':[[0,0]], 'nearSideLine':[[0,0]], 'farSideLine':[[0,0]]}
    for ln in self.linesID:
      lnType=ln[1]
      pos[lnType].append(np.array([ln[0][0]+ln[0][2], ln[0][1]+ln[0][3]])/2)
    for lnType in pos:
      if len(pos[lnType]) > 0:
        pos[lnType] = pos[lnType][1:]
      pos[lnType]=np.mean(pos[lnType], axis=0)
    return pos

  @property
  def lineTypeStdPos(self):
    pos = {'UpPitchLine': [[0,0]], 'halfway': [[0,0]], 'other': [[0,0]], '22L': [[0,0]], 'tryLineL':[[0,0]], 'tryLineR':[[0,0]], '22R':[[0,0]], 'nearSideLine':[[0,0]], 'farSideLine':[[0,0]]}
    for ln in self.linesID:
      lnType=ln[1]
      pos[lnType].append(np.array([ln[0][0]+ln[0][2], ln[0][1]+ln[0][3]])/2)
    for lnType in pos:
      if len(pos[lnType]) > 0:
        pos[lnType] = pos[lnType][1:]
      pos[lnType]=np.std(pos[lnType], axis=0)
    return pos


  def removeBadCrossPitchLines(self):
    for ln in self.linesID:
      lnT=ln[1]
      slope=ln[2][0]
      if lnT=='tryLineL' or lnT=='22L' or lnT=='tryLineR' or lnT=='22R':
        if np.abs(np.mean(slope) - np.mean(self.lineTypeSlopes[lnT])) > 0.05:
          ln[1]='other'
    self.lts = self.lineTypeSlopes
    for ln in self.linesID:
      lnT=ln[1]
      slope=ln[2][0]
      for typ in ['tryLineL','22L','tryLineR','22R']:
        if lnT=='other' and np.abs(ln[2][0]-np.mean(self.lts[typ]))<0.02:
          ln[1]=typ

  def removeGoalPosts(self):
    mupos=self.lineTypeMeanPos
    self.stdpos=self.lineTypeStdPos
    for ln in self.linesID:
      if ln[1]=='halfway':
        if self.stdpos['halfway'][0]>200:
          ln[1]='other'
        elif    self.lookingLeft and ln[0][0] - mupos['22L'][0] < 0:
          ln[1]='other'
        elif  self.lookingRight and  ln[0][0] - mupos['22R'][0] > 0:
          ln[1]='other'
    return self.linesID


  def checkPerspective(self):
    self.lookingLeft=False
    self.lookingRight=False
    self.ltsums=self.lineTypeLengthSum
    if self.ltsums['22L'] > self.ltsums['22R']*15:
      self.lookingLeft=True
      self.lookingRight=False
    elif self.ltsums['22R'] > self.ltsums['22L']*15:
      self.lookingLeft=False
      self.lookingRight=True
    if self.lookingLeft or self.lookingRight:
      for ln in self.linesID:
        if self.lookingLeft and (ln[1]=='22R' or ln[1]=='tryLineR'   ):
          ln[1]='other'
        elif self.lookingRight and (ln[1]=='22L' or ln[1]=='tryLineL'):
          ln[1]='other'
    self.muFSLSlope=np.mean(self.ltsums['farSideLine'])
    self.stdFSLSlope=np.std(self.ltsums['farSideLine'])

  def tryLineOrEndLine(self):
    delIndex=[]
    ptpYint=10000
    # i=0
    while ptpYint>150:
      self.ltYint = self.lineTypeYintercepts
      LmdYint    = np.median(self.ltYint['tryLineL'])
      RmdYint    = np.median(self.ltYint['tryLineR'])
      LptpYint = np.ptp(self.ltYint['tryLineL']) if not self.ltYint['tryLineL']==[] else 10000
      RptpYint = np.ptp(self.ltYint['tryLineR']) if not self.ltYint['tryLineR']==[] else 10000
      for ln in self.linesID:
        if ln[1]=='tryLineL' or ln[1]=='tryLineR':
          sign=ln[2][0]/np.abs(ln[2][0])
          thisYint=-ln[2][1]/ln[2][0]
          # thisSlope=ln[2][0]
          if ln[1]=='tryLineL':
            mdYint = LmdYint
            ptpYint = LptpYint
          elif ln[1]=='tryLineR':
            mdYint =  RmdYint
            ptpYint = RptpYint
          if ptpYint>150: break
          if (  sign*thisYint>sign*mdYint  ):
            # if len(np.where(self.linesID[:,1]==ln[1])[0]) > 7:
              ln[1]='other'
    return self.linesID


  def twenty2orTryLine(self):
    delIndex=[]
    ptpYint=10000
    # i=0
    while ptpYint>100:
      self.ltYint = self.lineTypeYintercepts
      LmdYint    = np.median(self.ltYint['22L'])
      RmdYint    = np.median(self.ltYint['22R'])
      LptpYint = np.ptp(self.ltYint['22L']) if not self.ltYint['22L']==[] else 10000
      RptpYint = np.ptp(self.ltYint['22R']) if not self.ltYint['22R']==[] else 10000
      for ln in self.linesID:
        if ln[1]=='22L' or ln[1]=='22R':
          sign=ln[2][0]/np.abs(ln[2][0])
          thisYint=-ln[2][1]/ln[2][0]
          # thisSlope=ln[2][0]
          if ln[1]=='22L':
            mdYint = LmdYint
            ptpYint = LptpYint
          elif ln[1]=='22R':
            mdYint =  RmdYint
            ptpYint = RptpYint
          if ptpYint>100: break
          if (  sign*thisYint>sign*mdYint  ):
              ln[1]='other'
    return self.linesID

  def tryLn22OrSideLn(self):
    delIndex=[]
    self.meanPos=self.lineTypeMeanPos
    self.ltsum=self.lineTypeLengthSum
    for ln in self.linesID:
      if ((ln[1]=='tryLineL' or ln[1]=='22L') and
          (self.ltsum['22L']+self.ltsum['tryLineL']<self.ltsum['22R']+self.ltsum['tryLineR']) and
          (self.meanPos[ln[1]][0]>self.shi[1]*0.35 and self.meanPos[ln[1]][1]>self.shi[0]*0.35)):
        if  self.isNearSideLine(ln[2][0], ln[2][1], ln):
          ln[1]='nearSideLine'
        elif self.isFarSideLine(ln[2][0], ln[2][1], ln):
          ln[1]='farSideLine'
        else:ln[1]='UpPitchLine'
      elif ((ln[1]=='tryLineR' or ln[1]=='22R') and
            (self.ltsum['22L']+self.ltsum['tryLineL']>self.ltsum['22R']+self.ltsum['tryLineR'])and
            (self.meanPos[ln[1]][0]<self.shi[1]*.65 and self.meanPos[ln[1]][1]>self.shi[0]*0.35)):
        if  self.isNearSideLine(ln[2][0], ln[2][1], ln):
          ln[1]='nearSideLine'
        elif self.isFarSideLine(ln[2][0], ln[2][1], ln):
          ln[1]='farSideLine'
        else:ln[1]='UpPitchLine'
    return self.linesID

  def inNonTryLineQuad(self, ln):
    return ((ln[1]=='tryLineL' and
                  (ln[0][0]>self.shi[1]*0.13 or ln[0][2]>self.shi[1]*0.13) and
                  (ln[0][1]>self.shi[0]*0 or ln[0][3]>self.shi[0]*0)) or
            (ln[1]=='tryLineR' and
                          (ln[0][0]<self.shi[1]*0.87 or ln[0][2]<self.shi[1]*0.87) and
                          (ln[0][1]>self.shi[0]*0 or ln[0][3]>self.shi[0]*0))
            )

  def purgeFSLs(self):
    for ln in self.linesID:
      if ln[1]=='farSideLine':
        if self.lookingLeft or self.lookingRight:
          if self.lookingLeft:
            sl22 = np.mean(self.lts['22L'])
            sign=-sl22/(np.abs(sl22))
          elif self.lookingRight:
            sl22 = np.mean(self.lts['22R'])
            sign=-sl22/(np.abs(sl22))
          if (ln[2][0]-sign*0.02)*(sl22)>0:
            ln[1]='UpPitchLine'

  def purgeNSLs(self):
    self.lts = self.lineTypeSlopes
    muslope=np.mean(self.lts['nearSideLine'])
    for ln in self.linesID:
      if ln[1]=='nearSideLine':
        if not (self.lookingLeft or self.lookingRight):
          if np.abs(muslope-ln[2][0])>0.02:
            ln[1]='UpPitchLine'
        if self.lookingLeft or self.lookingRight:
          if self.lookingLeft:
            sl22 = np.mean(self.lts['22L'])
            sign=-sl22/(np.abs(sl22))
          elif self.lookingRight:
            sl22 = np.mean(self.lts['22R'])
            sign=-sl22/(np.abs(sl22))
          if (ln[2][0]-sign*0.01)*(sl22)>0:
            ln[1]='UpPitchLine'

  def is22L(self, slope, C, ln):
    slope*=-1
    return (True and   0.27 <= (slope   )  and  (slope  ) <= 0.8)

  def is22R(self, slope, C, ln):
    return (True and   0.27 <= (slope   )  and  (slope   ) <= 0.8)

  def isTrylL(self, slope, C, ln):
    slope*=-1
    return (True and   0.09 <= (slope   )  and  (slope   ) <= 0.4)

  def isTrylR(self, slope, C, ln):
    return (True and   0.09 <= (slope   )  and  (slope   ) <= 0.4)

  def isNearSideLine(self, slope, C, ln):
    return (np.isclose(slope, 0, atol=0.26, equal_nan=False) and
            (ln[0][1]>self.shi[0]*0.75 or ln[0][3]>self.shi[0]*0.75))

  def isFarSideLine(self, slope, C, ln):
    return (np.isclose(slope, 0, atol=0.11, equal_nan=False) and
            (ln[0][1]<self.shi[0]*0.15 or ln[0][3]<self.shi[0]*0.15))


  def classifyLine0(self, ln):
    slope = ln[2][0]
    C = ln[2][1]
    if np.abs(slope) > 3: return "halfway"
    elif  self.is22L(slope,C,ln) :  return "22L"
    elif  self.is22R(slope,C,ln):  return "22R"
    elif  self.isTrylL(slope,C,ln):  return "tryLineL"
    elif  self.isTrylR(slope,C,ln):  return "tryLineR"
    elif  self.isFarSideLine(slope, C, ln) :      return "farSideLine"
    elif  self.isNearSideLine(slope,C, ln) :      return "nearSideLine"
    elif  np.isclose(slope, 0, atol=1.9e-1, equal_nan=False): return "UpPitchLine"
    else:
      return "other"

  def getPerspective(self, linesID):
    # print("hlf std",np.std(self.lineTypeSlopes['halfway']))
    self.nearHalfway=np.isclose(np.mean(self.lineTypeSlopes['nearSideLine']), 0, 0.05)

  def makeFarSideLine(self):
    x=-1000000
    if True:# len(np.where(self.linesID[:,1]=='farSideLine')[0])<10:
      mask = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)
      mask = cv2.inRange(mask, lowerb=-1, upperb=0)
      xl,yl,xr,yr=-1,-1,-1,-1
      if self.lookingLeft:
        for y in range(self.shi[0]):
          for x in range(self.shi[1])[::-1]:
            if mask[y][x]==0:
              xl=x
              yl=y+25
              break
          if xl==x:
            break
        for y in range(self.shi[0]):
            if mask[y][self.shi[1]-2]==0:
              xr=self.shi[1]-1
              yr=y+25
              break
      elif self.lookingRight:
        for y in range(self.shi[0]):
            if mask[y][0]==0:
              xl=0
              yl=y+25
              break
        for y in range(self.shi[0]):
          for x in range(self.shi[1]):
            if mask[y][x]==0:
              xr=x
              yr=y+25
              break
          if xr==x:
            break
      else:
        for y in range(self.shi[0]):
            if mask[y][0]==0:
              xl=0
              yl=y+20
              break
        for y in range(self.shi[0]):
            if mask[y][-1]==0:
              xr=self.shi[1]-1
              yr=y+20
              break
      pts=np.array([xl,yl,xr,yr])
      ln=[pts,'farSideLine']
      eq=self.lco.cartesianize(ln)
      newLn=np.array([pts,'farSideLine', eq, self.lco.dist(ln=pts)])
      if self.isFarSideLine(newLn[2][0], newLn[2][1], newLn):
        self.linesID=np.append(self.linesID, [newLn], axis=0)

  def classifyLines(self, lines, image):
    self.image=image
    self.shi=np.shape(image)
    self.linesID = np.empty(shape=(0,4))
    lineLengths = []
    for ln in lines:
      ln = [ln, "", self.lco.cartesianize([ln]), self.lco.dist(ln=ln)]
      lnType=str(self.classifyLine0(ln))
      ln[1] = lnType
      self.linesID = np.append(self.linesID, [ln], axis=0)
      self.linesID=np.array(self.linesID)
    self.ltsum=self.lineTypeLengthSum
    self.lengthPrioritizeLines(['22L','22R'])
    self.linesID=self.twenty2orTryLine()
    self.linesID=self.tryLn22OrSideLn()
    self.linesID=self.tryLineOrEndLine()
    self.lengthPrioritizeLines(['trylineL','trylineR'])
    self.checkPerspective()
    self.removeBadCrossPitchLines()
    self.checkPerspective()
    self.removeGoalPosts()
    self.checkPerspective()
    self.purgeFSLs()
    # self.makeFarSideLine()
    self.lengthPrioritizeLines(['nearSideLine','farSideLine', 'halfway'])
    self.purgeNSLs()
    return self.linesID

In [ ]:
class LineDrawer:
  def __init__(self):
    self.lco = LineCornerOperations()

  def colorSet(self):
    return {
        "tryLineL"      : (0,0,165),
        "22L"           : (0, 220, 165),
        "halfway"       : (90, 165, 255),
        "22R"           : (165, 0, 0),
        "tryLineR"      : (165, 165, 50),
        "UpPitchLine"   : (255,255,255),
        "nearSideLine"  : (50,200,220),
        "farSideLine"   : (200,220,50),
        "other"         : (90, 50, 90)
    }

  def drawLines(self, image, linesID, color=(0, 0, 0), thickness=2, metrics=False, lineColors=False, cartesianize=False):
    image = np.copy(image)
    if type(linesID[0][0])==np.float64:
      lll = np.empty(shape=(0,3))
      for ln in linesID:
        lll = np.append(lll, [[ln, 'other', []]], axis=0)
      linesID=lll
    for lnData in linesID:
      ln = lnData[0] if not np.shape(linesID)==(1,4) else  lnData
      # print(ln)
      text = f"{np.round(self.lco.getSlope(ln),2)}" if metrics else ""
      color = self.colorSet()[lnData[1]] if lineColors else color
      # lineTypes = np.unique(ln[1])
      colors = []
      pt1,pt2 = self.lco.points(ln)
      cv2.line(image,
                pt1=pt1,
                pt2=pt2,
                color=color,
                thickness=thickness)
      cv2.putText(img=image, text=text, org=(pt1[0]-5, pt1[1]+40),
                  fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.4, color=color,thickness=1)
      if cartesianize:
        # print(lnData)
        text=str(np.round(lnData[2],2))
        cv2.putText(img=image, text=text, org=(pt2[0]-200, pt2[1]+50),
                    fontFace=cv2.FONT_HERSHEY_TRIPLEX, fontScale=0.4, color=color,thickness=1)
    return image

In [ ]:
class LineCornerOperations:

  def cartesianize(self, ln):
    xs = ln[0][[0,2]]
    ys = ln[0][[1,3]]
    return np.polyfit(xs,ys, 1)


  def collinear(self, ln1, ln2, atol=1e-1):
    x1, y1, x2, y2, x3, y3, x4, y4 = ln1[0], ln1[1], ln1[2], ln1[3], ln1[0], ln1[1], ln1[2], ln1[3]
    a = np.abs(x1 * (y2 - y3) + x2 * (y3 - y1) + x3 * (y1 - y2))
    b = np.abs(x4 * (y2 - y3) + x2 * (y3 - y4) + x3 * (y4 - y2))
    # print(a,b)
    return  np.isclose(a, 0,  atol=atol) and np.isclose(b, 0, atol=atol)

  def parallel(self, ln1, ln2):
    return (((self.getSlope(ln1) - self.getSlope(ln2))**2)**0.5 <= 10**-10)

  def getSlope(self, ln):
    rise=ln[3] - ln[1]
    run = ln[2] - ln[0]+10**-100
    if run == 0: run=10**-100
    return rise/run

  def joinColLines(self, ln1, ln2):
    # print(ln1, ln2, '\n\n')
    xs = np.sort(np.append(ln1, ln2)[::2])
    ys = np.sort(np.append(ln1,ln2)[1::2])
    # print(xs,'\n\n', ys, '\n\n')
    return [xs[0], ys[0], xs[-1], ys[-1]]

  def getCollinears(self, line, lines):
      colLines = []
      for li in lines:
        ln =  li
        if self.collinear(line, ln):
          colLines.append(ln)
      return colLines

  def points(self, ln):
      x1 = int(ln[0])
      y1 = int(ln[1])
      x2 = int(ln[2])
      y2 = int(ln[3])
      return (x1, y1),(x2, y2)

  def dist(self, pts = (0, 0), ln=[]):
    if not np.array_equal(ln,[]):
      pts = (self.points(ln))
    tupsub = tuple(map(lambda i, j: i - j, pts[0], pts[1]))
    return cv2.norm(tupsub)

  def handleMainLines(self, lines):
    collines = []
    for ln in lines:
      collines.append(self.getColLines(ln, lines))
    # print(collines)

  def indicesOfUpPitchLine(self, linesID, of=""):
    ids = np.array([row[1] for row in linesID])
    if of=="":
      return np.where((ids == 'UpPitchLine') | (ids == 'nearSideLine'))[0]
    else:
      return np.where((ids == of))[0]

  def refine_lines(self, lines, minLenFactor=0.05):
    keepLines0, keepLines1, colLineCheckLines, colLineNumArr = [], [], [], []
    longestLen = np.max([self.dist(self.points(x)) for x in lines])
    shortestLen = np.min([self.dist(self.points(x)) for x in lines])
    defaultDist = longestLen*minLenFactor
    for li in lines:
        ln =  li
        pt1,pt2 = self.points(ln)
        dist = self.dist((pt1,pt2))
        if dist > defaultDist:
          keepLines0.append(ln)
          # colLines = self.getCollinears(ln, lines)
          # keepLines1+=colLines
    return keepLines0

  def lineIntersection(self, ln1, ln2):
    line1 = np.reshape(ln1, (2,-1))
    line2 = np.reshape(ln2, (2,-1))
    xdiff = (line1[0][0] - line1[1][0], line2[0][0] - line2[1][0])
    ydiff = (line1[0][1] - line1[1][1], line2[0][1] - line2[1][1])
    def det(a, b):
        return a[0] * b[1] - a[1] * b[0]
    div = det(xdiff, ydiff)
    if div == 0:
      raise Exception('lines do not intersect')
    d = (det(*line1), det(*line2))
    x = det(d, xdiff) / div
    y = det(d, ydiff) / div
    return [x, y]

  def findCrossPoints(self, linesID):
    crossPoints = [[],[]]
    lns = np.asarray(linesID)
    nsl = lns[np.where((lns[:,1]=="nearSideLine"))[0]][0]
    fsl = lns[np.where((lns[:,1]=="farSideLine"))[0]][0]
    cross = lns[np.where((  np.invert(lns[:,1]=="nearSideLine")) & ( np.invert(lns[:,1]=="farSideLine")))[0]]
    byLength = np.argsort([-self.dist(ln=ln) for ln in cross[:,0]])
    cross=cross[byLength][0:2]
    crossLineNames = {"tryLineL":0,"22L":1,"halfway":2,"22R":3,"tryLineR":4}
    if crossLineNames[cross[0][1]] > crossLineNames[cross[1][1]]:
      cross=np.flip(cross, axis=0)
    for ln in cross:
      crossPoints[0].append(self.lineIntersection(ln[0], nsl[0]) )
      crossPoints[0].append(self.lineIntersection(ln[0], fsl[0]) )
      crossPoints[1].append(ln[1])
      if len(crossPoints[0]) >=4:
        break
    return crossPoints

In [ ]:
class LineCornerDetector:
  def __init__(self, pd):
    self.ld = LineDrawer()
    self.lco = LineCornerOperations()
    self.lcl = LineClassifier(self.lco)
    self.lm = LineMerger(self.lco)
    self.pd=pd


  def lineImagePreprocessing(self, image):
    image = cv2.convertScaleAbs(image, alpha=1.2)
    # cv2_imshow(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    cv2_imshow(image)
    # image = cv2.convertScaleAbs(image)
    # cv2_imshow(image)
    return image

  def detectAndDrawLines(self, startImage, image, color1=(255,255,255), color2=(0,255,0)):
    dims =np.shape(image)
    # sys.exit(0)
    grey = self.lineImagePreprocessing(image)
    # cv2_imshow(grey)
    lines = np.rot90(
        FastLineDetector(do_merge=False, canny_aperture_size=7).detect(grey)[0], k=1)
    image0 = self.ld.drawLines(startImage, lines, color=color1, thickness=4, metrics=True, lineColors=False, cartesianize=False)
    cv2_imshow(image0)
    lines = self.lco.refine_lines(lines)
    image0 = self.ld.drawLines(startImage, lines, color=color1, thickness=4, metrics=True, lineColors=False, cartesianize=False)
    cv2_imshow(image0)
    # sys.exit(0)
    # # print(lines)
    linesID = self.lcl.classifyLines(lines, image)
    image0 = self.ld.drawLines(startImage, linesID, color=color1, thickness=4, metrics=False, lineColors=True, cartesianize=True)
    cv2_imshow(image0)
    mergedLinesID = self.lm.run(copy.deepcopy(linesID))
    image0 = self.ld.drawLines(startImage, mergedLinesID, color=color1, thickness=4, metrics=False, lineColors=True, cartesianize=True)
    cv2_imshow(image0)
    try:
      crossPoints = self.lco.findCrossPoints(mergedLinesID)
    except:
      print('couldn\'t find grid')
      crossPoints = []
    return startImage, crossPoints

  def applyLineDetector(self, pitchDetector, image, show=True, startImage=None):
    # cv2_imshow(image)
    if startImage is None:
      startImage=np.copy(image)
    # cv2_imshow(image)
    # sys.exit(0)
    image, crossPoints = self.detectAndDrawLines(startImage, image, color1=(150,50,50), color2=(255,255,100))
    # cv2_imshow(image)
    return image, crossPoints

  def refineTransform(self, startImage, image, color1=(255,255,255), color2=(0,255,0)):
    startImage=copy.deepcopy(image)
    dims =np.shape(image)
    # sys.exit(0)
    grey = self.lineImagePreprocessing(image)
    cv2_imshow(grey)
    lines = np.rot90(
        FastLineDetector(do_merge=False, canny_aperture_size=7).detect(grey)[0], k=1)
    lines = self.lco.refine_lines(lines)
    image0 = self.ld.drawLines(startImage, lines, color=color1, thickness=4, metrics=True, lineColors=False, cartesianize=False)
    cv2_imshow(image0)

In [ ]:
class PlayerDetector:
  def __init__(self):
    self.model = model

  def isolatePlayers(self, image):
    # hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    mask = np.zeros(shape=np.shape(image)[0:2])
    Grassmask = cv2.inRange(image, (20, 115,85), (80, 255, 150))
    notGrassmask = cv2.bitwise_not(Grassmask)
    notGrassImg = cv2.bitwise_and(image,  image, mask=notGrassmask)
    return notGrassImg

  def drawRectangles(self, image, dfResults, show=False):
    for index, row in dfResults.iterrows():
      imgResult = cv2.rectangle(image, (row['xmin'], row['ymin']), (row['xmax'], row['ymax']), (255, 0, 0), 2)
      imgResult = cv2.putText(image, row['name'],  (row['xmin'], row['ymin']), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    try:
      if show:
        cv2_imshow(imgResult)
    except:
      print("no players detected")
    return image

  def classifyPlayers(self, image, df):
    df['color'] = [(0,100,0) for r in df.iterrows()]
    # df[['Ireland', 'England', 'Bib', 'Ref']] = [[0,0,0,0] for r in df.iterrows()]
    for i, row in df.iterrows():
      x0,y0,x1,y1 = int(row['xmin']), int(row['ymin']), int(row['xmax']), int(row['ymax'])
      # print(image)
      # sys.exit(0)
      box = image[y0:y1,x0:x1]
      box = self.isolatePlayers(box)
      # cv2_imshow(box)
      bsh = np.shape(box)
      box = box[1*bsh[0]//6:3*bsh[0]//6, bsh[1]//3:int(2*bsh[1]//3)]
      boxPxls = cv2.countNonZero(cv2.cvtColor(box, cv2.COLOR_BGR2GRAY))
      # 111, 162, 52
      # 106, 160, 54

      # 17, 78, 11
      # 70, 189, 108 - jersey
      # cv2_imshow(box)
      jrsys = {
          'England' :[((250,250,250),  (255,255,255)),  4, (255,255,255)],
          'Ref'     :[((1,1,1),           (90,90,90)),4, (50,50,50)],
          'Bib'     :[((0,	247, 223), (135,255,255)), 5, (135,255,255)],
          'Ireland' :[((50, 90, 40),    (155,220,180)), 5, (1,99,55)]
      }
      pxlCounts=[]
      for j in jrsys:
        dst = cv2.inRange(box, jrsys[j][0][0], jrsys[j][0][1])
        jrsyPxls = cv2.countNonZero(dst)*jrsys[j][1]
        pxlCounts.append(jrsyPxls)
        # print(jrsyPxls)
      # print(jrsys)
      jr = max(range(len(pxlCounts)), key=pxlCounts.__getitem__)
      df.at[i,'name']=list(jrsys)[jr]
      df.at[i,'color']=list(jrsys.values())[jr][-1]
        # row[jrKey] = jrsyPxls
        # df['color']=jrsys[]
        # df.at[i,'name']  = 'Other'
        # df.at[i,'color']  = (20,20,180)
    # display(df)
    return df

  def runDetector(self, image):
    def fixY(y):
      return (y+i*sh[0]//ycut)
    def fixX(x):
      return (x+j*sh[1]//xcut)
    dfResults = pd.DataFrame()
    xcut,ycut=14,2
    sh = np.shape(image)
    for i in range(ycut):
      y0=(sh[0]*i)//ycut
      y1 = sh[0]*(i+1)//ycut
      for j in range(xcut):
        # print("x")
        x0 = (sh[1]*j)//xcut
        x1 = (sh[1]*(j+1))//xcut
        box = image[y0:y1,x0:x1]
        # cv2_imshow(box)
        results = self.model(box)
        df = results.pandas().xyxy[0]
        # print(df)
        # df = df.loc[(df['confidence']>0.26)]
        df = pd.concat([df, df.loc[(y1-df['ymax']<20) & (df['ymax'] - df['ymin'] < 80)], df.loc[(df['ymin']-y0<20) & (df['ymax'] - df['ymin'] < 80)]]).drop_duplicates(keep=False)
        df = pd.concat([df, df.loc[(x1-df['xmax']<10) & (df['xmax'] - df['xmin'] < 10)], df.loc[(df['xmax']-x0<10) & (df['xmax'] - df['xmin'] < 10)]]).drop_duplicates(keep=False)
        # print(df)
        if df.shape[0] > 0:
          df[['xmin','xmax']] = df[['xmin','xmax']].apply(fixX)
          df[['ymin','ymax']] = df[['ymin','ymax']].apply(fixY)
          dfResults = pd.concat([dfResults, df], ignore_index=True)
    # results = self.model(self.isolatePlayers(image))
    results = self.model(image)
    dfResults = pd.concat([dfResults, results.pandas().xyxy[0]], ignore_index=True).drop_duplicates(keep=False)
    dfResults = dfResults.loc[(dfResults['class']==0) & (dfResults['confidence']>0)]
    return dfResults

  def getPlayerCoords(self, image):
    # Inference
    image = cv2.convertScaleAbs(image, alpha=1.5, beta=-30)
    dfResults = self.runDetector(image)
    dfResults[['xmin', 'ymin', 'xmax','ymax']] = dfResults[['xmin', 'ymin', 'xmax','ymax']].astype(int)
    dfResults = self.classifyPlayers(image, dfResults)
    # display(dfResults)
    image = self.drawRectangles(copy.deepcopy(image), dfResults)
    dfResults['x'] = (dfResults[['xmin', 'xmax']].mean(axis=1))
    dfResults['y'] = (dfResults['ymax'])
    # sys.exit(0)
    return np.array([dfResults[['x', 'y', 'name','color']].to_numpy()]), image

In [ ]:
class PitchTransformer:
  def __init__(self):
    self.lco = LineCornerOperations()
    self.pitchImgShape = (800,1100)

  def drawPitch(self):
    pitch = np.full((self.pitchImgShape[1],self.pitchImgShape[0],3), (119,221,119), dtype=np.uint8)
    color=(240,240,240)
    pitch = cv2.rectangle(pitch, (50,50), (750, 1050), color, 4)
    #50m
    pitch = cv2.line(pitch, pt1=(50,550), pt2=(750,550), color=color, thickness=3)
    pitch = cv2.line(pitch, pt1=(50,830), pt2=(750,830), color=color, thickness=2)
    pitch = cv2.line(pitch, pt1=(50,270), pt2=(750,270), color=color, thickness=2)
    # 10m, 5m across
    pitch = cv2.line(pitch, pt1=(50,450), pt2=(750,450),   color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(50,650), pt2=(750,650),   color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(50,100), pt2=(750,100),   color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(50,1000), pt2=(750,1000), color=(150,150,150), thickness=1)
    # 5m Up, 15m up
    pitch = cv2.line(pitch, pt1=(100,50), pt2=(100,1050), color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(700,50), pt2=(700,1050), color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(200,50), pt2=(200,1050), color=(150,150,150), thickness=1)
    pitch = cv2.line(pitch, pt1=(600,50), pt2=(600,1050), color=(150,150,150), thickness=1)
    return pitch

  def makeImageCorners(self, lineTypes):
    pts = []
    w, l = 750, 1000
    if "tryLineL" in lineTypes:
      pts.append([50,l*0.05])
      pts.append([w,l*0.05])
    if "22L" in lineTypes:
      pts.append([50,l*0.27])
      pts.append([w,l*0.27])
    if "halfway" in lineTypes:
      pts.append([50,l*0.55])
      pts.append([w,l*0.55])
    if "22R" in lineTypes:
      pts.append([50,l*0.83])
      pts.append([w,l*0.83])
    if "tryLineR" in lineTypes:
      pts.append([50,l*1.05])
      pts.append([w,l*1.05])
    pts=np.asarray(pts)
    # print(pts)
    return np.float32(pts)

  def transformPitch(self, image, crossPoints, playerCoords):
    # cv2_imshow(image)
    shape = np.shape(image)
    pts1 = np.float32(crossPoints[0])
    pts2 = self.makeImageCorners(crossPoints[1])
    M = cv2.getPerspectiveTransform(pts1,pts2)
    image = cv2.warpPerspective(image,M,self.pitchImgShape)
    playerPts = cv2.perspectiveTransform(np.array([playerCoords[0][:,:2]], dtype=np.float32),M)
    try:
      playerCoords[0][:,0], playerCoords[0][:,1] = playerPts[0][:,0], playerPts[0][:,1]
    except Exception as e:
       print( str(e))
       playerCoords=[]
    return image, playerCoords

  def projectPlayerCoords(self, playerCoords):
    pitch = self.drawPitch()
    try:
      for p in playerCoords[0]:
          pitch = cv2.circle(pitch, (int(p[0]),int(p[1])), radius=10, color=p[3], thickness=-1)
    except:
      print("no players for pitch projection")
    return cv2.resize(pitch, ((524,720)))

  def mergeImageAndProjection(self, image, pitchProjection):
    sh = np.shape(pitchProjection)
    shi = np.shape(image)
    image = cv2.resize(image, (1280,720))
    return np.concatenate((image, pitchProjection), axis=1)

In [ ]:
class RugbyFieldRegistration:
  def __init__(self):
    self.frameCount=-1
    self.pld = PlayerDetector()
    self.lcd = LineCornerDetector(self)
    self.pt = PitchTransformer()
    self.imo = ImgMorphOperator()
    self.pi = PitchIsolator(self.imo)

  def detectPitch(self, rawFrame):
          self.frameCount+=1
          frame = rawFrame
          playersFrame, linesFrame = self.pi.isolatePitch(frame)
          playerCoords, plrDetectImage = self.pld.getPlayerCoords(copy.deepcopy(playersFrame))
          # return frame
          frame, crossPoints = self.lcd.applyLineDetector(self, linesFrame, startImage=plrDetectImage)
          # return frame
          # self.myWriter.cv2Writer.write(copy.deepcopy(cv2.resize(frame, (1600,1100))))
          # self.myWriter.writeFrame(frame)
          # return frame
          try:
            frameTr, playerCoords = self.pt.transformPitch(frame, crossPoints, playerCoords)
            # self.lcd.refineTransform(frameTr, frameTr)
            # cv2_imshow(frameTr)
            # sys.exit(0)
          except Exception as e:
            print("Failed pitch Transformation\n\n", e)
            playerCoords=[]
          pitchProjection = self.pt.projectPlayerCoords(playerCoords)
          frame = self.pt.mergeImageAndProjection(frame, pitchProjection)
          return frame

  def skipFrame(self, i, skipFrameRate):
      return not int(i % skipFrameRate)==0

  def flick_through_frames(self, myWriter):
    self.myWriter = myWriter
    i=-1
    while True:
      # Capture frame-by-frame
      ret, rawFrame = self.myWriter.cap.read()
      if ret==True:
        i+=1
        if not i % int(self.myWriter.start_fps/self.myWriter.output_fps)==0:
          continue
        if len(self.myWriter.skipFrames)>0 and i in self.myWriter.skipFrames:
          # self.myWriter.cv2Writer.write(rawFrame)
          continue
        print(f"frame {i}")
        if i in self.myWriter.skipFrames:
          continue
        if self.myWriter.quitEarly and i > self.myWriter.start_fps:
          break
        frame = self.detectPitch(rawFrame)
        # frame = cv2.resize(frame, (self.myWriter.frame_width, self.myWriter.frame_height))
        # cv2_imshow(frame)
        self.myWriter.writeFrame(frame)
        # sys.exit(0)
        # break
      else:
        print("Stream disconnected")
        break

class VideoWriter:
  def __init__(self):
      self.fourcc = cv2.VideoWriter_fourcc('M','P','4','V')
      # self.fourcc = cv2.VideoWriter_fourcc(*'MP4V')
      self.api=0

  def writeFrame(self, frame):
    self.cv2Writer.write(copy.deepcopy(cv2.resize(frame, (2560,1100))))

  def write_video(self):
    self.cv2Writer = cv2.VideoWriter(
        self.save_path,
        self.api,
        self.fourcc,
        self.output_fps,
       (2560,1100))
    self.vpc.flick_through_frames(self)
    self.cv2Writer.release()
    # cv2.destroyAllWindows()

  def run(self, cap, save_path, VideoProcessorClass, skipFrames=[], quitEarly=False, output_fps=None):
    """
    lnD - lineDetector
    imgPrp - image Preprocessing
    """
    self.vpc = VideoProcessorClass
    self.cap = cap
    self.save_path = save_path
    self.skipFrames=skipFrames
    self.num_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    self.frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    self.frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    self.start_fps = int(cap.get(cv2.CAP_PROP_FPS))
    self.output_fps = self.start_fps if output_fps is None else output_fps
    self.quitEarly = quitEarly
    self.write_video()

In [ ]:
# filePath = "/content/drive/My Drive/UL_Masters/rugby/"
# vid_cap = cv2.VideoCapture(filePath+"outputs/_40Frames_fullMatch/rawSidOn.mp4")
# # including frames 9, 10 to illustrate bad frames for field reg
# skipFrames =[]#[1,5,6,7,8,13,15,21,22,23,25,26,27,28,29,30,31,32,33,34,36,37,39]
#  # [0,2,4,3,   1,5,6,7,8,9,10,13,15,21,22,23,25,26,27,28,29,30,31,32,33,34,36,37,39]
# skipFrames = np.arange(0,38)
# pitchDetector = RugbyFieldRegistration()
# with warnings.catch_warnings():
#   VideoWriter().run(cap=vid_cap, save_path=filePath+"outputs/_40Frames_fullMatch/pitchTransformAndProjectionV2.mp4", VideoProcessorClass=pitchDetector, skipFrames=skipFrames)

In [ ]:
filePath = "/content/drive/My Drive/UL_Masters/rugby/"
vid_cap = cv2.VideoCapture(filePath+"outputs/playTrim/trim.mp4")
skipFrames = []#np.arange(0,59)#[30,45,60]#[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# skipFrames = np.arange(0,38)
pitchDetector = RugbyFieldRegistration()
with warnings.catch_warnings():
  VideoWriter().run(cap=vid_cap, save_path=filePath+"outputs/playTrim/projection.mp4", VideoProcessorClass=pitchDetector, skipFrames=skipFrames, output_fps=2)

In [ ]:
filePath = "/content/drive/My Drive/UL_Masters/rugby/"
vid_cap = cv2.VideoCapture(filePath+"outputs/playTrim/trim2.mp4")
skipFrames = []#np.arange(0,59)#[30,45,60]#[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# skipFrames = np.arange(0,38)
pitchDetector = RugbyFieldRegistration()
with warnings.catch_warnings():
  VideoWriter().run(cap=vid_cap, save_path=filePath+"outputs/playTrim/projection2.mp4", VideoProcessorClass=pitchDetector, skipFrames=skipFrames, output_fps=2)